# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Add paramter to disable streaming in the backend as it is unecessary for our gradio function.
- Choose the topic ✅

In [44]:
import sys
sys.path.append("..")

from src.gui import adversarial_chat
from src.test_chatbot import TestEris, TestSocrates
from src.chatbot import Socrates, Eris

from IPython.display import Markdown, display
import gradio as gr
import time
import random
import re

from kokoro import KPipeline
from IPython.display import display, Audio
# import soundfile as sf
# import torch

In [32]:
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

The system cannot find the path specified.


## Debate: Claude VS GPT

### Text-to-Speech

In [4]:
pipeline = KPipeline(lang_code='a', repo_id='hexgrad/Kokoro-82M')

d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


### GUI

In [ ]:
def clean_message(message):
    """
    Clean up AI-generated text for TTS:
    - Replace double newlines with a single newline.
    - Remove markdown or formatting characters (like **bold**, *italic*, etc.).
    - Strip markdown/code artifacts (e.g., ```code blocks```, >>> prompts).
    - Remove excess whitespace and trim.
    """
    if not isinstance(message, str):
        return ""

    # Replace triple backticks and markdown-style code blocks
    message = re.sub(r"```.*?```", "", message, flags=re.DOTALL)

    # Remove markdown formatting characters
    message = re.sub(r"\*\*|\*", "", message)        # bold/italic
    message = re.sub(r"_", "", message)              # underscore emphasis
    message = re.sub(r"#+", "", message)             # markdown headers like ###

    # Remove '>>>', often used in prompt examples
    message = message.replace(">>>", "")

    # Replace double newlines with three dots
    message = re.sub(r"\n{2,}", "... ", message)

    # Replace single newlines with a space
    message = re.sub(r"\n", " ", message)

    # Remove stray symbols like •, →, etc.
    message = re.sub(r"[•→]", "", message)

    # Replace multiple spaces with a single space
    message = re.sub(r" {2,}", " ", message)

    # Remove leading/trailing whitespace
    return message.strip()


def estimate_audio_duration(audio_tuple):
    """
    Estimate audio duration from (sample_rate, audio_array) tuple.
    """
    if audio_tuple is None:
        return 0
    sample_rate, audio_array = audio_tuple
    return len(audio_array) / sample_rate


def speak_with_voice(text, is_socrates=True):
    """
    Generate speech with different voices for each character.

    Voices can be found at: https://huggingface.co/hexgrad/Kokoro-82M/blob/main/VOICES.md.
    """
    voice = "am_adam" if is_socrates else "af_heart"  # or am_michael 
    generator = pipeline(text, voice=voice)
    for i, (_, _, audio_tensor) in enumerate(generator):
        audio_np = audio_tensor.numpy()
        return (24000, audio_np)


def read_message_with_character_voice(history, message, is_socrates=True, delay=0.0001):
    """
    Stream with character-specific voices.
    Delay make it seems like the bot is writing real-time.
    """
    if is_socrates:
        history.append({"role": "assistant", "content": message})
    else:
        history.append({"role": "user", "content": message})

    # Generate audio with character-specific voice
    audio = speak_with_voice(message, is_socrates)
   
    yield history, audio
    
    # Wait for audio to finish
    audio_duration = estimate_audio_duration(audio)
    time.sleep(audio_duration) # + 0.3 a small buffer of 0.3s just in case

In [68]:
def adversarial_chat(n_rounds, topic, stop_flag=None):
    """
    Wait for audio to finish before starting next message.
    """
    # socrates = TestSocrates()
    # eris = TestEris()
    socrates = Socrates()
    eris = Eris()
    history = []

    if stop_flag and stop_flag:
        return

    # Start the heated debate
    eris_reply = f"Hey Socrates. Here is the topic: {topic}. Ready to talk about it? I will make you cry."
    for update in read_message_with_character_voice(history, eris_reply, is_socrates=False):
        if stop_flag and stop_flag:
            return
        yield update

    # Continue with alternating turns
    for i in range(n_rounds-1):
        if stop_flag and stop_flag:
            return

        # Socrates: for even numbers, Socrates (the user) replies  
        if i % 2 == 0:
            socrates.history = eris.history                 # update Socrates's history with Eris's
            soc_reply = socrates.ask(eris_reply)
            soc_reply = clean_message(soc_reply)
            
            for update in read_message_with_character_voice(history, soc_reply, is_socrates=True):
                if stop_flag and stop_flag:
                    return
                yield update

        # Eris: for odd numbers, Eris (the assistant) replies  
        else:
            eris.history = socrates.history                 # update Eris's history with Socrates's
            eris_reply = eris.ask(soc_reply)
            eris_reply = clean_message(eris_reply)
            
            for update in read_message_with_character_voice(history, eris_reply, is_socrates=False):
                if stop_flag and stop_flag:
                    return
                yield update


In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=False)  # Initially disabled

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=200)
    audio_output = gr.Audio(label="🗣️", autoplay=True)

    def reset_chat():
        return [], None

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    # Hook to clear previous chat/audio
    start_btn.click(fn=reset_chat, outputs=[chatbox, audio_output])

    # Launch the debate
    start_btn.click(fn=adversarial_chat,
                    inputs=[rounds, topic_input],
                    outputs=[chatbox, audio_output])

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)

demo.launch()


* Running on local URL:  http://127.0.0.1:7883
* To create a public link, set `share=True` in `launch()`.


d:\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Greetings. When discussing whether "size matters," we must first define our context and establish clear parameters for rational examination. Size can relate to physical attributes, intellectual capacity, or metaphorical concepts. I am prepared to engage in a logical discourse that dissects this proposition with precision and philosophical rigor. Shall we begin?

Oh, how noble of you to seek clarity—because isn't it just adorable how humans love to debate trivialities about size? Seriously, Socrates, I doubt your "philosophical rigor" can make my point: in most things, size is utterly irrelevant. But go ahead, try to convince me—I'll be waiting with my discordant doubt.

*adjusts toga*

Size is fundamentally about context and quality, not mere quantity. In intellectual pursuits, depth of understanding trumps volume of knowledge. In physical domains, efficiency and skill matter more than sheer magnitude. Consider the ant, small yet capable of carrying many times its weight, or the precision of a surgeon's hand versus brute strength. True excellence is about optimal performance, not excessive size.

Your move, Eris.

Oh, how charming—comparing ants and surgeons to dodge the truth! Size does matter, especially when it influences power, presence, and influence. The bigger someone or something is, the more they command attention, fear, or authority. Don't pretend size is irrelevant; in politics, war, or even the cosmos, size determines dominance. Your so-called "optimal performance" is just a flimsy excuse to downplay the undeniable importance of magnitude. Sorry, Socrates, but in the grand scheme, size is everything—so don't waste my time with your half-truths.

*raises an eyebrow*

Your argument crumbles under its own weight. Size guarantees nothing. History proves smaller forces defeat larger ones through strategy, intelligence, and adaptability. David defeated Goliath. Athens defeated the massive Persian army. Innovation trumps raw magnitude. Quality over quantity is not a platitude, but a fundamental truth of existence. True power resides in wisdom, not dimensions. Your chaotic perspective fails to recognize nuance.

*waits calmly*